<a href="https://colab.research.google.com/github/suzunotree/Python-for-Data-Science-and-AI-final-project-group-4-/blob/main/Group_4_Final_group_project_Python_for_Data_Science_and_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/suzunotree/Python-for-Data-Science-and-AI-final-project-group-4-/tree/main

In [ ]:
! pip install transformers datasets

In [ ]:
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Step 1: Load the dataset
# Load the dataset from Hugging Face's dataset repository
dataset = load_dataset("sroecker/Multi-lingual_Translation_Instruct-langsplit", split="train")
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/434 [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/250M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2877582 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'src_lang', 'dest_lang'],
    num_rows: 2877582
})


In [ ]:
# Convert Dataset to a DataFrame
df = pd.DataFrame(dataset)
print("Sample data:")
print(df.head())

Sample data:
                                         instruction  \
0  You are an AI translator, translate the follow...   
1  You are an AI translator, translate the follow...   
2  You are an AI translator, translate the follow...   
3  You are an AI translator, translate the follow...   
4  You are an AI translator, translate the follow...   

                                               input  \
0  After these things, Jesus walked in Galilee: f...   
1  You scorn me, Alexis, and ask not what I am'ho...   
2  the corn not only was being consumed by so gre...   
3  Thus spoke Hannibal; but the men of Rome, on t...   
4  No further did Faith, the Virtues queen, bear ...   

                                              output src_lang dest_lang  
0  post haec ambulabat Iesus in Galilaeam non eni...  english     latin  
1  Despectus tibi sum nec, qui sim, quaeris, Alex...  english     latin  
2  frumenta non solum tanta multitudine iumentoru...  english     latin  
3  Talibus hic Po

In [ ]:
print(dataset.features)
print(dataset[0])

{'instruction': Value(dtype='string', id=None), 'input': Value(dtype='string', id=None), 'output': Value(dtype='string', id=None), 'src_lang': Value(dtype='string', id=None), 'dest_lang': Value(dtype='string', id=None)}
{'instruction': 'You are an AI translator, translate the following from english to latin.', 'input': 'After these things, Jesus walked in Galilee: for he would not walk in Judea, because the Jews sought to kill him.', 'output': 'post haec ambulabat Iesus in Galilaeam non enim volebat in Iudaeam ambulare quia quaerebant eum Iudaei interficere', 'src_lang': 'english', 'dest_lang': 'latin'}


In [ ]:
# Check the unique language pairs
print("\nUnique Source Languages:", df['src_lang'].unique())
print("Unique Destination Languages:", df['dest_lang'].unique())


Unique Source Languages: ['english' 'dutch,']
Unique Destination Languages: ['latin' 'urdu' 'chinese' 'portuguese' 'korean' 'russian']


In [ ]:
# Filter English to Chinese
english_to_chinese = df[(df['src_lang'] == 'english') & (df['dest_lang'] == 'chinese')]

In [ ]:
# Subset for demonstration
subset_df = english_to_chinese[['input', 'output']].head(10)

print("\nFiltered Dataset (English to Chinese):")
print(subset_df)


Filtered Dataset (English to Chinese):
                                                 input  \
155  On 14 October 2013, Foreign Minister Erlan Idr...   
156  Richard's misfortunes seemed to follow him int...   
160  STK has been deployed by many mobile operators...   
161  Due to the semantics of some programming langu...   
167  In the Wa language, spoken in the borderlands ...   
170  Another ship on September 16 reported similar ...   
173                     It used two different rockets.   
177  Its continuing mission: to explore strange new...   
179  Uematsu did not only show up at Anime Boston, ...   
183  Like most of pop music, its songs tend to be w...   

                                                output  
155          2013年10月14日,外交部长厄兰·伊德里索夫会见了乌克兰外长列昂里德·库扎哈。  
156                                      理查的逃脱为他带来了好处。  
160  全球众多移动运营商已使用STK技术部署了众多应用,这些应用通常是基于菜单式操作,提供诸如手机...  
161  由于某些编程语言的语义,编译器生成的代码允许在线程A执行完变量的初始化之前,更新变量并将其指...  
167         在云南省与掸邦边境地区的佤语中,称呼中国人的词为

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch

In [ ]:
# Load the pre-trained mBART model and tokenizer
model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

In [ ]:
# Set the target language
tokenizer.src_lang = "en_XX"  # English source language
def translate_text(text, target_lang="zh_CN"):
  encoded_input = tokenizer(text, return_tensors="pt")
  generated_tokens = model.generate(
    **encoded_input,
    forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"])
  tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [ ]:
# Function to Translate Text
def translate_text(text, target_lang="zh_CN"):  # Chinese target
    encoded_input = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    generated_tokens = model.generate(**encoded_input, forced_bos_token_id=tokenizer.lang_code_to_id[target_lang])
    return tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]


In [ ]:
# Apply Translation on Subset
subset_df['Translated_Text'] = subset_df['input'].apply(lambda x: translate_text(x, target_lang="zh_CN"))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
# Display Results
print("\nTranslation Results:")
print(subset_df[['input', 'output', 'Translated_Text']])


Translation Results:
                                                 input  \
155  On 14 October 2013, Foreign Minister Erlan Idr...   
156  Richard's misfortunes seemed to follow him int...   
160  STK has been deployed by many mobile operators...   
161  Due to the semantics of some programming langu...   
167  In the Wa language, spoken in the borderlands ...   
170  Another ship on September 16 reported similar ...   
173                     It used two different rockets.   
177  Its continuing mission: to explore strange new...   
179  Uematsu did not only show up at Anime Boston, ...   
183  Like most of pop music, its songs tend to be w...   

                                                output  \
155          2013年10月14日,外交部长厄兰·伊德里索夫会见了乌克兰外长列昂里德·库扎哈。   
156                                      理查的逃脱为他带来了好处。   
160  全球众多移动运营商已使用STK技术部署了众多应用,这些应用通常是基于菜单式操作,提供诸如手机...   
161  由于某些编程语言的语义,编译器生成的代码允许在线程A执行完变量的初始化之前,更新变量并将其指...   
167         在云南省与掸邦边境地区的佤语中,称呼中国人的词为Hox/Hawx,发音为/

In [ ]:
!pip install nltk
import nltk
from nltk.translate.bleu_score import corpus_bleu

In [ ]:
# Evaluate Translation Quality using BLEU Score
references = [[ref] for ref in subset_df['output'].tolist()] # List of ground truth translations
hypotheses = subset_df['Translated_Text'].tolist()  # Model-generated translations

# Compute BLEU Score
bleu_score = corpus_bleu(references, hypotheses) # Order of arguments swapped to match NLTK BLEU convention.
print("\nBLEU Score for English to Chinese Translation:", bleu_score)


BLEU Score for English to Chinese Translation: 0.3210862217004309
